In [1]:
#comparing relaxed, not relaxed, repack input True and repack input False 
#using Pool to speed things up to be less awful 
#adapting the multiprocessing supported pyrosetta scripts to run single mutation scan across all of a protein 

import logging
logging.basicConfig(level=logging.INFO)
import pandas
import seaborn
import matplotlib
import Bio.SeqUtils
import Bio.Data.IUPACData as IUPACData
import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as dist_packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as dist_rosetta_scripts
import pyrosetta.distributed.tasks.score as dist_score

from multiprocessing import Pool
import itertools
import pandas as pd

        
import dask.distributed
import numpy as np

from pyrosetta import *
import random
from pyrosetta.toolbox import cleanATOM
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.pack.task import *
from pyrosetta.rosetta.protocols import *
from pyrosetta.rosetta.core.select import *


import math
import time
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

pyrosetta.init()

INFO:pyrosetta.rosetta:Found rosetta database at: /home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pyrosetta-2017-py3.6-linux-x86_64.egg/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python36.linux 2019.04+release.fd666910a5edac957383b32b3b4c9d10020f34c1 2019-01-22T15:55:37] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python36.linux r208 2019.04+release.fd66691 fd666910a5edac957383b32b3b4c9d10020f34c1 http://www.pyrosetta.org 2019-01-22T15:55:37
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pyrosetta-2017-py3.6-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:core.init: 'RNG device' seed mo

In [2]:
#wider jupyter cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
#functions for mutation, unbinding and dG
#10/30/20- changing so that it does not repack 

#my version - mutate a position to Alanine 
#code adapted from PyRosetta tutorials 
def pack(pose, strRep, amino, scorefxn):

    # Select Mutate Position
    mut_posi = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector()
    mut_posi.set_index(strRep)
    # print(pyrosetta.rosetta.core.select.get_residues_from_subset(mut_posi.apply(pose)))

    # Select Neighbor Position
    nbr_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    nbr_selector.set_focus_selector(mut_posi)
    nbr_selector.set_include_focus_in_subset(True)
    # print(pyrosetta.rosetta.core.select.get_residues_from_subset(nbr_selector.apply(pose)))

    # Select No Design Area
    not_design = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(mut_posi)
    # print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_design.apply(pose)))

    # The task factory accepts all the task operations
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

    # These are pretty standard
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

    # Disable Packing
    prevent_repacking_rlt = pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()
    prevent_subset_repacking = pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(prevent_repacking_rlt,
                                                                                                 nbr_selector, True)
    tf.push_back(prevent_subset_repacking)

    # Disable design
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
        pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), not_design))

    # Enable design
    aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
    aa_to_design.aas_to_keep(amino)
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(aa_to_design, mut_posi))

    # Create Packer
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.task_factory(tf)

    # Perform The Move
    if not os.getenv("DEBUG"):
        packer.apply(pose)
    return pose
        
def unbind(pose, partners, packInput):
    #10/30/20: changing pack input to try and preserve inital HBNets for analysis per Ajasja's rec
    #switching to InterfaceAnalyzerMover (IAM) 
    iamMover = analysis.InterfaceAnalyzerMover()
    iamMover.set_interface("A_B")
    iamMover.set_pack_input(packInput)
    iamMover.set_pack_separated(True)
    iamMover.apply(pose)
    interfaceData = iamMover.get_all_data()
    return interfaceData.interface_hbonds, interfaceData.delta_unsat_hbonds, interfaceData.hbond_E_fraction, iamMover.get_interface_dG()


#run a mutation where those in the list are changed to Alanine 
def runRandomMutation(packedPose, numbers, flag):
    #return total h bonds, unsat h bonds, fraction inferace E from h bonds 
    pose = packedPose
    #mutate and unbind 
    if len(numbers) != 0:
        strRep = ",".join(numbers)
        scorefxn = get_score_function()
        scores = []
        times = []
        #mTDG
        original = pose.clone()
        mutantPose = pack(original, strRep, 'A', scorefxn)
        return unbind(mutantPose, "A_B", flag)
    else:
        #run the wt 
        strRep = ",".join(numbers)
        scorefxn = get_score_function()
        scores = []
        #wTDG
        original = pose.clone()
        return unbind(original, "A_B", flag)


def allResidues(chainALen, chainBLen):
    #creates all possible indexes for position selection
    totalLen = chainALen + chainBLen
    totalList = list(range(1, totalLen + 1))
    return [str(x) for x in totalList]


#changing so instead of mutating a group of top x%, select top x% then mutate each indiv.
#takes N other random not in the top x% list and mutates them too 
def runAlaScanOnDimer(structureName, rowA, rowB, status, endName = ".relax.pdb"):
    
    #setting up for starmapping of wt, mt, and randoms 
    runAll = []
    runAll.append([])#for running wt

    #for eah row, get the positions of a above the percentile
    pose = pose_from_pdb("./test_set_structures/" + structureName + endName)
    
    #select random residues not in the top X% 
    allPositions =allResidues(rowA, rowB)
    allPosList = [[x] for x in allPositions]
    runAll = runAll + allPosList

    #attach pose info & run mutations
    work = [(pose, pair, status) for pair in runAll]
    with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
        with Pool() as p:
            logging.info("mutating")
            mutations = p.starmap(runRandomMutation, work)

    #get wt info 
    wtNumberHbonds = mutations[0][0]
    wtNumberUnsatHBonds = mutations[0][1]
    wtFractionEnergyHBonds = mutations[0][2]
    wtdG = mutations[0][3]

    #mutation info 
    for i in range(1, len(mutations)):
        otherHBonds.append( mutations[i][0] -  wtNumberHbonds)
        otherHBondsUnsat.append( mutations[i][1] -  wtNumberUnsatHBonds)
        otherHBondEnergyFraction.append( mutations[i][2] - wtFractionEnergyHBonds)
        otherddGs.append( mutations[i][3] - wtdG)
            
    return  otherHBonds, otherHBondsUnsat, otherHBondEnergyFraction, otherddGs

In [4]:
#running False and True for repack input & comparing results 
#opening csv scores 

csvName = "test_set.csv" # "Pos95thPercentileWorkingVersion.csv"
scoresCSV = pd.read_csv(csvName)
print (scoresCSV.shape)

(480, 7)


In [10]:
#run the ddG procedure on all dimers in the test set 

startTime = time.time()
for ind, row in scoresCSV.iterrows():
    print ("Done, time elapsed: ", time.time() - startTime)
    print ("Starting: ", ind)
    otherHBondsFalseNR, otherHBondsUnsatFalseNR, otherHBondEnergyFractionFalseNR, otherddGsFalseNR = runAlaScanOnDimer(row['structure'],row['lenA'], row['lenB'], False, ".pdb")
    saveName = row['structure']
    otherHBondsFalseNR = np.array(otherHBondsFalseNR)
    np.save("./NRFalseScores/" + row['structure'] + "_hbonds.npy", otherHBondsFalseNR)
    otherHBondsUnsatFalseNR = np.array(otherHBondsUnsatFalseNR)
    np.save("./NRFalseScores/" + row['structure'] + "_unsat_hbonds.npy", otherHBondsUnsatFalseNR)
    otherHBondEnergyFractionFalseNR = np.array(otherHBondEnergyFractionFalseNR)
    np.save("./NRFalseScores/" + row['structure'] + "_hbonds_energy.npy", otherHBondEnergyFractionFalseNR)
    otherddGsFalseNR = np.array(otherddGsFalseNR)
    np.save("./NRFalseScores/" + row['structure'] + "_ddg.npy", otherddGsFalseNR)
    print (time.time() - startTime)
    startTime = time.time()
    print ("---------------------------------------------------------------------")

INFO:rosetta:core.import_pose.import_pose: File './test_set_structures/redesigned_closed_5_7_9_9middlesbobby_1_1_S_394469.pdb_middle1.pdb-5_7_9_9middlesscott_1_1_S_390782.pdb_middle1.pdb-bobby_1_1_S_435474_padded_0001_0002.pdb' automatically determined to be of type PDB


Done, time elapsed:  0.001180410385131836
Starting:  0


INFO:root:mutating


NotImplementedError: __cereal_getstate__ requires pyrosetta '--serialization' build.